# <font color="blue"> MBA em Ciência de Dados</font>
# <font color="blue">Técnicas Avançadas para Captura e Tratamento de Dados</font>

## <font color="blue"> Matriz Documento $\times$ Palavras - Bag of Words</font>
    
## <font color="blue">Avaliação</font>

**Material Produzido por Luis Gustavo Nonato**<br>
**Cemeai - ICMC/USP São Carlos**
---

Os exercícios abaixo fazem uso da coleção de documentos presente no diretório `DocCol2` contido no arquivo <font style="font-family: monaco"> DocCol.zip</font>, o qual pode ser baixado do Moodle.

In [1]:
import glob
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np
from collections import Counter

### Exercício 1)
Armazene os documentos disponíveis no diretório `DocCol2` em um dicionário onde a chave é o nome do arquivo e o valor é a string contida no arquivo. O documento contendo a string com o maior número de caracteres é:

a) gr7<br>
b) au2 <br>
c) ch5<br>
d) au8

In [2]:
# Lendo o documento como uma string
import glob  # glob é um pacote que permite varrer arquivos e 
             # diretorios utilizando sintaxe linux

files = glob.glob("DocCol2/*")

docs = {}
for fname in files:
    with open(fname,'r') as f:
        key = fname.split('\\')[-1]
        docs[key] = f.read() 
        


In [3]:
len(docs['au10'])

4617

In [4]:
tam={}
for i in docs.keys():
    tam[i]=len(docs[i])



In [5]:
import pandas as pd
df=pd.DataFrame(tam.values(),index=tam.keys(),columns=['tamanho'])
df.sort_values('tamanho',ascending=False).head(1)

,tamanho
au2,11672


In [6]:
import operator
max_key = max(tam.items(), key=operator.itemgetter(1))[0]
print(max_key)

au2


### Exercício 2)
Crie um dicionário chamado `docsXwords` onde as chaves são os nomes dos arquivos e os valores são as listas de palavras do documento correspondente. As palavras em cada uma das listas devem ser constituídas apenas por letras do alfabeto, estarem lexicamente normalizadas e conterem mais que 1 caracter. Qual o documento cuja lista de palavras  resultante possui o **maior** número de palavras (lexicamente normalizadas) repetidas:

a) gr22<br>
b) ch30<br>
c) au1<br>
d) au8

In [37]:
import string
stop_words = stopwords.words('english')
docsXwords = {}
for key, value in docs.items():
    words = nltk.word_tokenize(value)
    words = [w.lower() for w in words if w.isalpha() and w not in string.ascii_letters]
    #words = [w for w in words if w not in stop_words]  #ele nao pede stopwords
    words = [PorterStemmer().stem(w) for w in words]
    docsXwords[key] = words

In [61]:
dupes={}
for i in docsXwords.keys():
    y=0
    for x in docsXwords[i]:
        if docsXwords[i].count(x)>1:
            y=y+1
    dupes[i]=y
            
        
    

In [63]:
df1=pd.DataFrame(dupes.values(),index=dupes.keys(),columns=['tamanho'])

In [65]:
df1.sort_values('tamanho',ascending=False).head()

,tamanho
au2,1356
ch30,1317
gr23,1218
ch24,1202
ch44,1198


In [24]:
len(docsXwords['gr24'])

575

### Exercício 3)
Utilizando as listas de palavras do dicionário `docsXwords`, quais as três palavras que mais aparecem na coleção de documentos:

a) the, is, of<br>
b) that, is, of<br>
c) the, of, to <br>
d) to, is, of

### Exercício 4)
Qual o documento cuja lista de palavras possui o **menor** número de "stop words"? Quantas "stop words" aparecem neste documento:

a) gr5 com 47 "stop words"<br>
b) gr17 com 47 "stop words"<br>
c) gr5 com 37 "stop words"<br>
d) gr17 com 37 "stop words"

**Dica**: Crie um dicionário a partir de docsXwords onde a chave é o nome do documento e o valor o número de stop words no documento.

In [72]:
stop={}
for i in docsXwords.keys():
    y=0
    for x in docsXwords[i]:
        if x in stop_words:
            y=y+1
    stop[i]=y

    
df2=pd.DataFrame(stop.values(),index=stop.keys(),columns=['tamanho'])           

In [73]:
df2.sort_values('tamanho',ascending=True).head()

,tamanho
gr5,47
gr17,68
gr12,69
gr7,99
gr13,129


### Exercício 5) 
Utilize o dicionário `docsXwords` para construir
uma matriz Documentos $\times$ Palavras para a coleção de documentos do diretório `DocCol2`. Utilizando a distância cosseno, qual é o documento mais parecido com o documento 'ch7':

a) ch8<br>
b) ch16<br>
c) ch5<br>
d) au8

In [110]:
import glob
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
from nltk import word_tokenize
from nltk.collocations import *

# lista das stop words
stop_words = stopwords.words('english')

files = glob.glob("DocCol2/*")  # vamos utilizar os documento que estão no diretório DocCol3 

ddocs = {}
for fname in files:
    key = fname.split('\\')[-1]  # rotulo do documento pega o que vem antes da "/"
    with open(fname,'r') as f:  # obtendo a lista de palavras processadas de cada documento
        doc = f.read()
        # obtendo palavras do documento
        words = nltk.word_tokenize(doc) 
        # removendo simbolos
        words = [w.lower() for w in words if w.isalpha()] 
        # removendo stop words
        words = [w for w in words if w not in stop_words]
        #bigramas
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        finder = BigramCollocationFinder.from_words(words)
        for i in finder.nbest(bigram_measures.raw_freq,20):
            words.append(i[0].lower()+' '+i[1].lower())
        # stemming
        words = [PorterStemmer().stem(w) for w in words]
        # calculando a frequência de palavras
        ddocs[key] = dict(Counter(words))

In [114]:
# constrindo a lista de todas as palavras
# precisamos desta lista para definir as colunas do DataFrame Documentos X Palavras
corpus = []

for v in ddocs.values():
    corpus.extend(list(v.keys()))  # juntando as listas de 
                                   # palavras de cada documento
    
corpus = list(set(corpus)) # removendo palavras repetidas
print(corpus[:10])

['unsupport', 'horac', 'alvin writ', 'deadlin', 'static', 'casual', 'secur', 'airbag look', 'catholic theologian', 'ontario']


In [115]:
# construindo um DataFrame onde os rótulos das colunas são
# as palavras e os das linhas os documentos
import pandas as pd
import numpy as np

rows = list(ddocs.keys())

BoW = pd.DataFrame(data=np.zeros((len(rows),len(corpus))),
                   columns=corpus,index=rows,dtype=int)

for k in ddocs.keys():
    BoW.loc[k,list(ddocs[k].keys())] = list(ddocs[k].values())

BoW


,unsupport,horac,alvin writ,deadlin,static,casual,secur,airbag look,catholic theologian,ontario,...,weigh,see differ,walk,immaculatam,secular,boe,jordan,smiley,covet,seventh
au1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
au10,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
au11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
au12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
au13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gr5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gr6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
gr7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gr8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
BoW

,unsupport,horac,alvin writ,deadlin,static,casual,secur,airbag look,catholic theologian,ontario,...,weigh,see differ,walk,immaculatam,secular,boe,jordan,smiley,covet,seventh
au1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
au10,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
au11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
au12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
au13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gr5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gr6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
gr7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gr8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
from scipy import spatial

result = 1- spatial.distance.cosine(list(BoW.loc['ch7']), list(BoW.loc['ch7']))
result

1.0

In [120]:
from scipy.spatial import distance
x=1
di={}
for i in BoW.index:
    di[i] = distance.cosine(list(BoW.loc['ch7']), list(BoW.loc[i]))
   
df3=pd.DataFrame(di.values(),index=di.keys(), columns=['cos'])
df3.sort_values('cos').head(20)
    
    

,cos
ch7,0.000000
ch10,0.602170
ch17,0.676281
ch3,0.680037
ch14,0.698558
ch11,0.699704
ch44,0.717344
ch33,0.731803
ch48,0.742438
ch25,0.759648
